In [1]:
import requests
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd
import os

In [2]:
def get_ip():
    response = requests.get('https://api64.ipify.org?format=json').json()
    return response["ip"]


def get_location():
    ip_address = get_ip()
    response = requests.get(f'https://ipapi.co/{ip_address}/json/').json()
    location_data = {
        "ip": ip_address,
        "city": response.get("city"),
        "region": response.get("region"),
        "country": response.get("country_name")
    }
    return location_data

In [3]:
print(get_location())

{'ip': '208.85.234.131', 'city': 'Springfield', 'region': 'Oregon', 'country': 'United States'}


In [21]:
def get_zip(x):
    response = requests.get(f'https://ipapi.co/{x}/json/').json()
    res = response.get('postal')
    if type(res) is str:
        return int(res)
    else:
        return None

In [4]:
ip_address = get_ip()

In [18]:
ip_address = '73.240.110.67'

In [19]:
response = requests.get(f'https://ipapi.co/{ip_address}/json/').json()

In [44]:
type(response.get('postal')) is str

True

In [2]:
path = r'T:\DCProjects\Support\Lane'

In [3]:
dat = pd.read_csv(os.path.join(path, 'TransitPlanningforRuralLaneCounty.csv'))

In [4]:
dat.columns

Index(['RespondentID', 'CollectorID', 'StartDate', 'EndDate', 'IPAddress',
       'ZipCode', 'Area'],
      dtype='object')

In [78]:
dat.loc[:, 'ZipRev'] = dat.copy()['IPAddress'].apply(lambda x: get_zip(x))

In [32]:
def revZipCode(x, y):
    if x == y:
        res = 0
    else:
        res = 1
    return res

In [34]:
dat.columns

Index(['RespondentID', 'CollectorID', 'StartDate', 'EndDate', 'IPAddress',
       'ZipCode', 'Area', 'ZipRev'],
      dtype='object')

In [48]:
dat.loc[:, 'Diff'] = dat.copy()[['ZipCode', 'ZipRev']].apply(lambda row: revZipCode(row.ZipCode, row.ZipRev), axis=1)

In [38]:
dat.shape

(194, 9)

In [4]:
excluded = [114239498080, 114240368347, 114240387977, 114241348088]

In [19]:
excludedIP = ['208.85.234.131', '71.94.234.141', '73.15.153.115', '73.240.110.67']

In [22]:
list(map(lambda x: get_zip(x), excludedIP))

[97477, 97440, 94501, 97401]

In [5]:
114239498080 in dat.RespondentID.unique()

False

In [18]:
[rid for rid in excluded if rid in dat.RespondentID.unique()] 

[]

In [6]:
df = dat[['RespondentID', 'ZipCode']].groupby(['ZipCode']).agg('count').reset_index()

In [7]:
df['ZipCode'] = df['ZipCode'].apply(str)

In [8]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

zip_sql = '''
SELECT 
zipcode AS ZipCode,
mailcity AS City,
Shape.STAsBinary() AS geometry
FROM dbo.ZIPCode;
'''

In [9]:
def readZipCode():
    ZipCode = gpd.GeoDataFrame.from_postgis(zip_sql, engine, geom_col='geometry')
    ZipCode.crs = "EPSG:2914"
    return ZipCode

In [10]:
zip_shp = readZipCode()

In [11]:
data = zip_shp.merge(df, on='ZipCode', how='left')

In [12]:
data.rename(columns={"RespondentID": "IDcount"}, inplace=True)

In [13]:
data.loc[data.IDcount.isnull(), 'IDcount'] = 0

In [14]:
data.to_file(os.path.join(path, 'Survey_Count.shp'))

In [15]:
[zipc for zipc in df.ZipCode.unique() if zipc not in zip_shp.ZipCode.unique()]

['97467']

In [17]:
[zipc for zipc in zip_shp.ZipCode.unique() if zipc not in df.ZipCode.unique()]

['97324',
 '97390',
 '97431',
 '97434',
 '97446',
 '97451',
 '97452',
 '97454',
 '97455',
 '97456',
 '97461',
 '97480',
 '97492',
 '97493',
 '97438']

In [24]:
[zipc for zipc in ['97477', '97440', '94501', '97401'] if zipc not in zip_shp.ZipCode.unique()]

['97440', '94501']

In [18]:
data

,ZipCode,City,geometry,IDcount
0,97324,Alsea,"POLYGON ((4080351.085 968607.066, 4079414.210 ...",0.0
1,97390,Tidewater,"POLYGON ((4034530.460 971416.316, 4034424.585 ...",0.0
2,97401,Eugene,"POLYGON ((4250897.121 879922.502, 4250954.413 ...",15.0
3,97402,Eugene,"POLYGON ((4215595.835 913587.191, 4210310.585 ...",11.0
4,97403,Eugene,"POLYGON ((4243940.960 878874.691, 4243643.210 ...",2.0
5,97404,Eugene,"POLYGON ((4232869.585 911578.566, 4232818.335 ...",5.0
6,97405,Eugene,"POLYGON ((4225032.634 876260.298, 4222884.056 ...",11.0
7,97412,Blachly,"POLYGON ((4108370.585 904688.128, 4117618.460 ...",1.0
8,97419,Cheshire,"POLYGON ((4146113.585 943843.316, 4143751.960 ...",1.0
9,97424,Cottage Grove,"POLYGON ((4228670.585 808120.816, 4224575.835 ...",4.0
